# Assignment 2 :

Gowtham is an algo trader who wants to perform some stock market analysis on ICICI bank. He wants
to store the real-time stock market data of ICICI bank for this week in his database. For each day, he
wants to store the data from 11.00 AM to 2.00 PM. However, he cannot afford to pay for a real-time
API, so he decides to use the Python Yahoo Finance library which provides real-time data around a 15-
minutes delay.

__Write a Python program that will store 15-minute candle data of ICICI bank for every 15 minutes from
11.15 AM to 2.15 PM daily for a week. The program should use the Yahoo Finance library to retrieve
the real-time data and store it in the database. Due to the 15-minute delay in the Yahoo Finance library,
the program should start logging at 11.15 AM to capture data for the 11.00 AM time interval.__

__Note:__
    
You should use MongoDB as a database and python APS Scheduler library for this task. The ticker for
ICICI bank is "ICICIBANK.NS" and you can obtain stock information from Yahoo Finance using the
yfinance library

# Install Relevant Libraries:

__pip install yfinance__

__pip install apscheduler__

__pip install pymongo__


In [ ]:
# importing relevent libraries
import yfinance as yf
from apscheduler.schedulers.blocking import BlockingScheduler
from pymongo import MongoClient
from datetime import datetime

In [ ]:
# Function to fetch and store ICICI Bank's real-time data
def fetchStoreData():
    
    # current timestamp
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    # Fetching real-time data with a 15-minute delay using yfinance library
    icici_data = yf.Ticker("ICICIBANK.NS").history(period='1d', interval='15m')
    
    # Establish connection to MongoDB server
    client = MongoClient('mongodb://localhost:27017/') 
    
    # Accessing / creating a database named 'stock_data'
    db = client['stock_data'] 
    
    # Accessing / creating a collection named 'icici_bank'
    collection = db['icici_bank']  
    
    # Store the fetched data in MongoDB with the timestamp
    data_dictionary = icici_data.to_dict(orient='records') 
    
    for record in data_dictionary:
        
        # Adding current timestamp to each record
        record['timestamp'] = current_time 
        collection.insert_one(record)  
    
    print(f"Data stored in MongoDB at {current_time}")

In [ ]:
# Create a scheduler
scheduler = BlockingScheduler()

In [ ]:
# Schedule the job to run every 15 minutes from 11:15 AM to 2:15 PM
scheduler.add_job(fetchStoreData, 'interval', minutes=15, start_date='2023-10-23 11:15:00', end_date='2023-10-23 14:15:00')

In [ ]:
# Start the scheduler
scheduler.start()